In [30]:
def solution(n, k, cmd):
    stack = []
    answer = ['O'] * n
    table = [0 for _ in range(n)]

    for c in cmd:
        if c[0] == 'C': # 삭제
            stack.append(k)
            answer[k] = 'X'
            table = table[:k-1] + table[k+1:] # 삭제

            if k > len(table) - 1:
                k -= 1

        elif c[0] == 'D': # 커서 아래로
            temp = c[2]
            k += int(temp)
        elif c[0] == 'U': # 커서 위로
            temp = c[2]
            k -= int(temp)

        elif c[0] == 'Z': # 다시 제자리로
            pos = stack.pop()
            table = table[:pos-1] + [0] + table[pos-1:] # 삭제
            answer[pos] = 'O'

            if pos <= k:
                k += 1

    return ''.join(answer)

In [31]:
solution(8, 2, ["D 12", "C", "U 3", "C", "D 4", "C", "U 2", "Z", "Z"])

'OOOXOOOO'

In [27]:
solution(8, 2, ["D 2","C","U 3","C","D 4","C","U 2","Z","Z","U 1","C"])

'OOXOXOOO'

In [20]:
import copy


def solution(n, k, cmd):
    stack = []
    idx_stack = []
    answer = ['O'] * n
    table = [0 for _ in range(n)]

    for c in cmd:
        if c[0] == 'C': # 삭제
            stack.append(copy.deepcopy(table))
            idx_stack.append(k)
            answer[k] = 'X'
            table = table[:k-1] + table[k+1:] # 삭제

            if k > len(table) - 1:
                k -= 1

        elif c[0] == 'D': # 커서 아래로
            temp = c[2]
            k += int(temp)
        elif c[0] == 'U': # 커서 위로
            temp = c[2]
            k -= int(temp)

        elif c[0] == 'Z': # 다시 제자리로
            table = stack.pop()
            pos = idx_stack.pop()
            answer[pos] = 'O'

            if pos <= k:
                k += 1

    return ''.join(answer)

In [21]:
solution(8, 2, ["D 2", "C", "U 3", "C", "D 4", "C", "U 2", "Z", "Z"])

'OXOOXXOO'